# Abundance Matching with `hmf` python package

In [1]:
%matplotlib inline

In [2]:
from halotools.sim_manager import CachedHaloCatalog
halocat = CachedHaloCatalog(simname='bolplanck', redshift=0.5)

hostmask = halocat.halo_table['halo_upid'] == -1
hosts = halocat.halo_table[hostmask]
hosts.sort('halo_mvir')

In [3]:
from hmf.hmf import MassFunction
mf = MassFunction(Mmin=10, Mmax=16, z=halocat.redshift)

In [4]:
from scipy.integrate import quad

def interpolated_dndlog10M(log10m):
    return 10**np.interp(log10m, np.log10(mf.m), np.log10(mf.dndlog10m))

def hmf_log10_cumnd(log10_mass):
    return np.log10(quad(interpolated_dndlog10M, log10_mass, np.inf)[0])

Compare cumulative number density in simulation to `hmf` package

In [5]:
comoving_volume = np.prod(halocat.Lbox)

for n in (100, 500, 5000, 50000):
    log10_cumnd_sim = np.log10(n/comoving_volume)
    log10_mass = np.log10(hosts[-n]['halo_mvir'])
    log10_cumnd_hmf = hmf_log10_cumnd(log10_mass)
    print("At log10(Mvir) = {0:.2f}, log10(N(>M)) = ({1:.2f}, {2:.2f})".format(log10_mass, log10_cumnd_sim, log10_cumnd_hmf))

At log10(Mvir) = 14.08, log10(N(>M)) = (-5.19, -5.09)
At log10(Mvir) = 13.74, log10(N(>M)) = (-4.49, -4.46)
At log10(Mvir) = 13.03, log10(N(>M)) = (-3.49, -3.47)
At log10(Mvir) = 12.10, log10(N(>M)) = (-2.49, -2.47)
